In [1]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

C:\Users\Sai Priya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Sai Priya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Sai Priya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Sai Priya\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Futu

In [8]:
import os
os.getcwd()

'C:\\Users\\Sai Priya\\Desktop\\internship1'

In [9]:
os.chdir("C:/Users/Sai Priya/Desktop/internship1")

In [10]:
data = pd.read_csv("stack-overflow-data.csv")

In [11]:
data.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [12]:
data['tags'].value_counts()

android          2000
sql              2000
php              2000
mysql            2000
java             2000
.net             2000
css              2000
jquery           2000
asp.net          2000
ios              2000
c                2000
javascript       2000
ruby-on-rails    2000
c++              2000
c#               2000
objective-c      2000
python           2000
angularjs        2000
iphone           2000
html             2000
Name: tags, dtype: int64

In [13]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 32000
Test size: 8000


In [14]:
train_posts = data['post'][:train_size]
train_tags = data['tags'][:train_size]

test_posts = data['post'][train_size:]
test_tags = data['tags'][train_size:]

In [15]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [16]:
tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [17]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [18]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [19]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (32000, 1000)
x_test shape: (8000, 1000)
y_train shape: (32000, 20)
y_test shape: (8000, 20)


In [20]:
batch_size = 32
epochs = 10

In [21]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


Train on 28800 samples, validate on 3200 samples
Epoch 1/10
28800/28800 [==============================] - 11s 383us/step - loss: 0.9729 - accuracy: 0.7178 - val_loss: 0.6149 - val_accuracy: 0.8109
Epoch 2/10
28800/28800 [==============================] - 11s 367us/step - loss: 0.5478 - accuracy: 0.8247 - val_loss: 0.5956 - val_accuracy: 0.8097
Epoch 3/10
28800/28800 [==============================] - 10s 364us/step - loss: 0.4571 - accuracy: 0.8495 - val_loss: 0.5887 - val_accuracy: 0.8116
Epoch 4/10
28800/28800 [==============================] - 11s 382us/step - loss: 0.3824 - accuracy: 0.8731 - val_loss: 0.6240 - val_accuracy: 0.8044
Epoch 5/10
28800/28800 [==============================] - 11s 367us/step - loss: 0.3224 - accuracy: 0.8932 - val_loss: 0.6436 - val_accuracy: 0.8047
Epoch 6/10
28800/28800 [==============================] - 10s 360us/step - loss: 0.2720 - accuracy: 0.9102 - val_loss: 0.6743 - val_accuracy: 0.7959
Epoch 7/10
28800/28800 [==============================] 

In [23]:
model.save("model.h5")

In [24]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

8000/8000 [==============================] - 0s 54us/step
Test score: 0.775711920440197
Test accuracy: 0.799875020980835


In [25]:
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_posts.iloc[i][:50], "...")
    print('Actual label:' + test_tags.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

jquery( tr_selector ) is removing the style values ...
Actual label:jquery
Predicted label: jquery

web reference in wcf  in wcf if we use webreferenc ...
Actual label:.net
Predicted label: .net

fragment transaction custom animation - android  i ...
Actual label:android
Predicted label: android

why can i not use the namespace directive in c++ s ...
Actual label:c++
Predicted label: c++

generating password reset link in c# .net  i must  ...
Actual label:.net
Predicted label: c#

php 5.3 $this versus php 5.4  i am calling a membe ...
Actual label:php
Predicted label: php

undefined method `request_uri  actiondispatch  i w ...
Actual label:ruby-on-rails
Predicted label: ruby-on-rails

my table view is not showing data  i am using a ta ...
Actual label:iphone
Predicted label: iphone

update panel not working correctly   i have added  ...
Actual label:asp.net
Predicted label: asp.net

floating objects in html  is it possible to make a ...
Actual label:html
Predicted label: html

